In [1]:
from pathlib import Path
import pandas as pd

In [2]:
MODEL_CKPT_DIR = 'model_params_big_test'
# MODEL_CKPT_DIR = 'res_balanced_accuracy'
# MODEL_CKPT_DIR = 'res_f1'

MODEL_DATA_FILE = Path(MODEL_CKPT_DIR) / 'data.csv'

In [3]:
# > AVERAGE FILES HERE
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_all_query_pairs.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-5_query_pairs.csv'

# > GOOD FILES HERE
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-5_query-23_nonquery-5539_pairs.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-5_query-23_nonquery-5539_embedded.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-20_query-23_nonquery-5539_embedded.csv'
DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-50_query-23_nonquery-5539_embedded.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-50_query-23_nonquery-5539_pairs.csv'

# > FILES W/INNER RUCLIP AS RANKER
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-30_query-2_nonquery-6_embedded.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-20query-23_nonquery-5539_embedded_sbert=all-distilroberta-v1_clip=siamese_contrastive.pt.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-20_query-2_nonquery-6_embedded_sbert=all-distilroberta-v1_clip=siamese_contrastive.pt_final-embs.csv'
# DATA_FILE = 'data/tables_OZ_geo_5500/processed/tabular_OZ_geo_5500_top-50_query-23_nonquery-5539_embedded_sbert=all-distilroberta-v1_clip=siamese_contrastive.pt_final-embs.csv'

df_all = pd.read_csv(DATA_FILE)
df_model = pd.read_csv(MODEL_DATA_FILE)
# df_all.columns.tolist()

In [4]:
pd.set_option('display.max_colwidth', None)

keywords = r'карта'
kw_mask = df_model.name_first.str.contains(keywords, case=False, regex=True)

df_model_domain = df_model[
    kw_mask

    # & (df_model.label == 1)
    # & (df_model.label == 0)
]

# for idx, row in df_model_domain.iterrows():
#     print(row.name_first)
#     print(row.sku_first)

#     print(row.name_second)
#     print(row.sku_second)
#     print('-' * 50)
#     print()

# pd.reset_option('display.max_colwidth')

In [7]:
from sentence_transformers import SentenceTransformer
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# SBERT_MODEL = 'ai-forever/ru-en-RoSBERTa'
# BATCH_SIZE = 512

SBERT_MODEL = 'all-distilroberta-v1'
BATCH_SIZE = 512

# BATCH_SIZE = 128
# SBERT_MODEL = 'sentence-transformers/gtr-t5-base'

sbert = SentenceTransformer(SBERT_MODEL, device=DEVICE)

In [8]:
# Define batch size for processing

# Extract options text
options_first = df_model_domain['options_first'].tolist()
options_second = df_model_domain['options_second'].tolist()

# Generate embeddings for options in batches
options_first_embeddings = sbert.encode(options_first, batch_size=BATCH_SIZE, show_progress_bar=True)
options_second_embeddings = sbert.encode(options_second, batch_size=BATCH_SIZE, show_progress_bar=True)

# Add embeddings to dataframe
df_model_domain['options_first_embedding'] = list(options_first_embeddings)
df_model_domain['options_second_embedding'] = list(options_second_embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_269834/3771414387.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model_domain['options_first_embedding'] = list(options_first_embeddings)
/tmp/ipykernel_269834/3771414387.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model_domain['options_second_embedding'] = list(options_second_embeddings)


In [57]:
# Calculate cosine similarity between options embeddings
from sklearn.metrics.pairwise import cosine_similarity
options_similarities = [cosine_similarity([emb1], [emb2])[0][0] 
                        for emb1, emb2 in zip(options_first_embeddings, options_second_embeddings)]
# Fix SettingWithCopyWarning by using .loc
df_model_domain.loc[:, 'options_similarity'] = options_similarities

In [75]:
# drop identical pairs
df_model_domain = df_model_domain[df_model_domain['sku_first'] != df_model_domain['sku_second']]

In [76]:
# Extract height and width from options text
import re

# Function to extract height and width from options text
def extract_height_width(options_text):
    # Look for height pattern like "Высота предмета: 107 см"
    height_match = re.search(r'Высота.*?: (\d+)', options_text, re.IGNORECASE)
    height = int(height_match.group(1)) if height_match else None
    
    # Look for width pattern like "Ширина предмета: 157 см"
    width_match = re.search(r'Ширина.*?: (\d+)', options_text, re.IGNORECASE)
    width = int(width_match.group(1)) if width_match else None

    # Look for combined pattern like "157x107"
    combined_match = re.search(r'(\d+)\s*[xх*]\s*(\d+)', options_text, re.IGNORECASE)
    if combined_match:
        width = int(combined_match.group(1))
        height = int(combined_match.group(2))

    return height, width


# Function to process each group
def process_group(group):
    # Extract height and width from first product options (reference)
    reference_options = group['options_first'].iloc[0]
    ref_height, ref_width = extract_height_width(reference_options)
    group['width_first'] = ref_width
    group['height_first'] = ref_height

    # Create new columns with height and width extraction results
    group.loc[:, 'height_second'] = group['options_second'].apply(
        lambda x: extract_height_width(x)[0]).astype('Int64')
    group.loc[:, 'width_second'] = group['options_second'].apply(
        lambda x: extract_height_width(x)[1]).astype('Int64')
    return group

# Apply the processing function to each group
grouped = df_model_domain.groupby('sku_first', group_keys=False)
processed_df = grouped.apply(process_group, include_groups=True).dropna()
processed_df.head(1)

/tmp/ipykernel_269834/1423736244.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_df = grouped.apply(process_group, include_groups=True).dropna()


,balance_first,sales_first,rating_first,final_price_first,comments_first,description_first,name_first,options_first,sku_first,has_video_first,...,name_sim,img_sim,label,options_first_embedding,options_second_embedding,options_similarity,width_first,height_first,height_second,width_second
22,1,1,5,432,95,"Карта мира настенная ""Мир"" 1,0х0,7м, м-б 1:34 000 000. Физическая карта мира напечатана одним листом на плотном мелованном картоне, без ламинации. Карта настенная прекрасно впишется в любое помещение и станет приятным подарком и для ребенка и для взрослого. Небольшой размер карты так же позволяет поместить ее на рабочий стол под стекло или под любой другой прозрачный материал. Карта мира для детей без ламинации, а значит она станет идеальным фотофоном для семейных и интерьерных фотосъемок, поскольку не дает бликов. Так же издания без ламинации позволяют рисовать или делать пометки на карте, что поможет в работе или развитии творческих способностей - карта развития ребенка. Современное учебное пособие для изучения географии и окружающего мира, атлас мира для школьников, для детей младшего и среднего школьного возраста. Карта мира для детей на стену прекрасный подарок для мальчика или девочке. Универсальная карта на стену станет ярким элементом декора в оформлении квартиры или дома, полезной деталью дизайна детской комнаты ребенка или гостиной, познавательной зоной в офисе или рабочем кабинете, вызывающей интерес. Скретч карта мира украсит детскую комнату или любую стену. Карта мира - нужный и интересный подарок к школе на 1 сентября, на выпускной ребенку, любимой женщине и полезное развлечение для всей семьи. Этот постер станет отличным помощником в учебе и школьных буднях для вашего ребенка, а также идеально подойдет для подростков для учебы в школе, университете, вузе, колледже, для работы. Цветная карта будет полезной стеновой декорацией в любом другом учреждении образования. В нашем ассортименте вы найдете также другие карты, такие как карта мира физическая и политическая, новая большая карта России настенная для детей, карта звездного неба и созвездий, карта дорог России с городами для путешественников, карта животных, карта животный и растительный мир, картина на стену, карта земли, карта для школы, постер космическое пространство и детский атлас мира. Закажите уже сегодня и украсьте свою стену красивым и информативным изображением нашей Родины - настенной географической картой мира! Изучайте мир вместе с АТЛАС ПРИНТ. Цветом показан рельеф суши и морского дна. Обозначены основные населенные пункты. Карта настольная, плакат на стену, картина на стену, декор на стену","Географическая физическая карта Мира ""Мир"" 100х70 см",Высота предмета: 70 см. Ширина предмета: 100 см. Тип карты: Географическая; детские карты; настенная. Вид карты: физическая; настольная карта; Интерьерная. Масштаб: 1:34000000. Страна производства: Россия. Комплектация: Карта Мира настенная 70х100 см.,64354900,0,...,0.83,0.61,1,"[0.032793194, -0.030637827, -0.0064851665, 0.019776396, -0.0633618, -0.035394456, 0.02328478, -0.020472685, -0.044235308, 0.016656656, 0.034888934, -0.025758406, -0.004444477, 0.0013730659, 0.016152378, -0.046341356, -0.021061925, -0.01838166, 0.025725948, 0.03079035, 0.02375403, 0.079863995, -0.00379466, -0.0037534677, -0.02411043, -0.0019409277, -0.0014351468, -0.0060945717, -0.13618548, 0.052619815, -0.050256092, 0.019624474, -0.03792269, 0.008710592, -0.028665915, 0.026701672, 0.046598956, -0.031180745, -0.0092269275, 0.07243504, -0.011656352, 0.0117158275, 0.05128237, 0.00022830546, -0.001957942, 0.035116617, 0.0005101067, -0.05734167, 0.02815501, -0.0029705737, -0.011123582, -0.02172256, -0.027522175, 0.0059820563, -0.0045746122, 0.042848255, 0.007714739, -0.12709941, 0.05282511, 0.032889668, -0.01632066, -0.03742703, -0.038455207, 1.9617113e-05, 0.019173514, -0.0492984, 0.006510193, 0.053324174, -0.00033854725, -0.049684227, 0.012852187, -0.029684111, 0.012068731, 0.014603281, 0.0071551227, -0.028081007, -0.04592802, 0.0

In [77]:
# Get embeddings for height and width
import numpy as np
from tqdm import tqdm

# Use a single template for the second embeddings
HW_TEMPLATE = 'height: {height}, width: {width}'
HW_TEMPLATE = '[{height}, {width}]'

# Initialize lists to store concatenated embeddings
width_height_embeddings_first = []
width_height_embeddings_second = []

# Process the dataframe in batches
for start_idx in tqdm(range(0, len(processed_df), BATCH_SIZE), desc="Processing batches"):
    # Get the batch of rows
    batch = processed_df.iloc[start_idx:start_idx + BATCH_SIZE]

    # Extract height and width for _first and _second products
    heights_first = batch['height_first'].tolist()
    widths_first = batch['width_first'].tolist()
    heights_second = batch['height_second'].tolist()
    widths_second = batch['width_second'].tolist()

    # Compute embeddings for _first products using an inline f-string
    embeddings_first = sbert.encode(
        [f'[{height}, {width}]' for height, width in zip(heights_first, widths_first)], 
        batch_size=BATCH_SIZE, show_progress_bar=False
    )
    width_height_embeddings_first.extend(embeddings_first)

    # Compute embeddings for _second products using the template with keyword arguments
    embeddings_second = sbert.encode([
        HW_TEMPLATE.format(height=height, width=width) if not np.isnan(height) and not np.isnan(width)
        else '[nan, nan]'
        for height, width in zip(heights_second, widths_second)
    ], batch_size=BATCH_SIZE, show_progress_bar=False)
    width_height_embeddings_second.extend(embeddings_second)

# Add the computed embeddings as new columns in the dataframe
processed_df['width_height_embedding_first'] = width_height_embeddings_first
processed_df['width_height_embedding_second'] = width_height_embeddings_second

Processing batches: 100%|██████████| 1/1 [00:03<00:00,  3.40s/it]


In [78]:
# Compute separate height and width embeddings for first and second products,
# then compute an aggregated embedding (concatenation of height and width embeddings)

height_embeddings_first = []
width_embeddings_first = []
agg_embeddings_first = []

height_embeddings_second = []
width_embeddings_second = []
agg_embeddings_second = []

for start_idx in tqdm(range(0, len(processed_df), BATCH_SIZE), desc="Processing batches"):
    batch = processed_df.iloc[start_idx:start_idx + BATCH_SIZE]
    
    # Extract height and width values for first and second products
    heights_first = batch['height_first'].tolist()
    widths_first = batch['width_first'].tolist()
    heights_second = batch['height_second'].tolist()
    widths_second = batch['width_second'].tolist()
    
    # Create stringified inputs (using "nan" if missing)
    height_strs_first = [str(h) if pd.notnull(h) else "nan" for h in heights_first]
    width_strs_first = [str(w) if pd.notnull(w) else "nan" for w in widths_first]
    height_strs_second = [str(h) if pd.notnull(h) else "nan" for h in heights_second]
    width_strs_second = [str(w) if pd.notnull(w) else "nan" for w in widths_second]
    
    # Compute embeddings for first products
    emb_height_first = sbert.encode(height_strs_first, batch_size=BATCH_SIZE, show_progress_bar=False)
    emb_width_first = sbert.encode(width_strs_first, batch_size=BATCH_SIZE, show_progress_bar=False)
    emb_agg_first = [np.concatenate([h, w]) for h, w in zip(emb_height_first, emb_width_first)]
    
    height_embeddings_first.extend(emb_height_first)
    width_embeddings_first.extend(emb_width_first)
    agg_embeddings_first.extend(emb_agg_first)
    
    # Compute embeddings for second products
    emb_height_second = sbert.encode(height_strs_second, batch_size=BATCH_SIZE, show_progress_bar=False)
    emb_width_second = sbert.encode(width_strs_second, batch_size=BATCH_SIZE, show_progress_bar=False)
    emb_agg_second = [np.concatenate([h, w]) for h, w in zip(emb_height_second, emb_width_second)]
    
    height_embeddings_second.extend(emb_height_second)
    width_embeddings_second.extend(emb_width_second)
    agg_embeddings_second.extend(emb_agg_second)

# Add the computed embeddings as new dataframe columns
processed_df['height_embedding_first'] = height_embeddings_first
processed_df['width_embedding_first'] = width_embeddings_first
processed_df['agg_embedding_first'] = agg_embeddings_first

processed_df['height_embedding_second'] = height_embeddings_second
processed_df['width_embedding_second'] = width_embeddings_second
processed_df['agg_embedding_second'] = agg_embeddings_second

Processing batches: 100%|██████████| 1/1 [00:02<00:00,  2.91s/it]


In [113]:
# Apply extraction to the selected SKU group
SELECTED_SKU_IDX = 1

unique_skus = processed_df['sku_first'].unique()
selected_sku = unique_skus[SELECTED_SKU_IDX]
selected_group = processed_df.groupby('sku_first').get_group(selected_sku).copy()  # Use .copy() to avoid SettingWithCopyWarning
selected_group.shape, selected_group.label.unique()

((34, 49), array([0, 1]))

In [115]:
# Get final ranking table for similarities
from sklearn.metrics.pairwise import cosine_similarity

ref_height, ref_width = selected_group[['height_first', 'width_first']].iloc[0]

# Compute Euclidean distance for all items if reference dimensions exist
if ref_height and ref_width:
    print(f"\nReference product dimensions: Height {ref_height} cm, Width {ref_width} cm")
    
    # Calculate Euclidean distance to reference height and width for all rows
    selected_group.loc[:, 'euclidean_distance'] = selected_group.apply(
        lambda row: ((row['height_second'] - ref_height) ** 2 + 
                     (row['width_second'] - ref_width) ** 2) ** 0.5 
        if pd.notnull(row['height_second']) and pd.notnull(row['width_second']) else None, 
        axis=1
    )
    
    # Sort by Euclidean distance
    sorted_group = selected_group.sort_values('euclidean_distance')
    
    # Display sorted results
    print("\nProducts sorted by Euclidean distance to reference dimensions:")
else:
    print("\nReference dimensions are not available.")
    
# Compute ranks by Euclidean distance and similarity without ties
if 'euclidean_distance' in selected_group.columns:
    selected_group['rank_by_distance'] = selected_group['euclidean_distance'].rank(method='first', ascending=True)
else:
    selected_group['rank_by_distance'] = None

if 'options_similarity' in selected_group.columns:
    selected_group['rank_by_similarity'] = selected_group['options_similarity'].rank(method='first', ascending=False)
else:
    selected_group['rank_by_similarity'] = None

# Compute similarity for width/height embeddings
if 'width_height_embedding_first' in selected_group.columns and 'width_height_embedding_second' in selected_group.columns:
    selected_group['wh_emb_sim'] = selected_group.apply(
        lambda row: cosine_similarity(
            [row['width_height_embedding_first']], 
            [row['width_height_embedding_second']]
        )[0][0] if not pd.isnull(row['width_height_embedding_first']).any() and not pd.isnull(row['width_height_embedding_second']).any() else None,
        axis=1
    )
    # Compute ranks for width/height embedding similarity
    selected_group['rank_by_wh_emb_sim'] = selected_group['wh_emb_sim'].rank(method='first', ascending=False)
else:
    selected_group['wh_emb_sim'] = None
    selected_group['rank_by_wh_emb_sim'] = None

# Compute similarity for aggregated embeddings and their ranks
if 'agg_embedding_first' in selected_group.columns and 'agg_embedding_second' in selected_group.columns:
    selected_group['agg_emb_sim'] = selected_group.apply(
        lambda row: cosine_similarity(
            [row['agg_embedding_first']], 
            [row['agg_embedding_second']]
        )[0][0] if row['agg_embedding_first'] is not None and row['agg_embedding_second'] is not None else None,
        axis=1
    )
    selected_group['rank_by_agg_emb_sim'] = selected_group['agg_emb_sim'].rank(method='first', ascending=False)
else:
    selected_group['agg_emb_sim'] = None
    selected_group['rank_by_agg_emb_sim'] = None

# Compute and print rank correlation between different ranking metrics
if 'rank_by_distance' in selected_group.columns and 'rank_by_similarity' in selected_group.columns:
    rank_corr_distance_similarity = selected_group[['rank_by_distance', 'rank_by_similarity']].corr(method='spearman').iloc[0, 1]
    print(f"\nSpearman rank correlation between distance and similarity ranks: {rank_corr_distance_similarity:.4f}")
else:
    print("\nRank columns are not available for correlation computation between distance and similarity.")

if 'rank_by_distance' in selected_group.columns and 'rank_by_wh_emb_sim' in selected_group.columns:
    rank_corr_distance_emb_sim = selected_group[['rank_by_distance', 'rank_by_wh_emb_sim']].corr(method='spearman').iloc[0, 1]
    print(f"Spearman rank correlation between distance and width/height embedding similarity ranks: {rank_corr_distance_emb_sim:.4f}")
else:
    print("\nRank columns are not available for correlation computation between distance and width/height embedding similarity.")

if 'rank_by_distance' in selected_group.columns and 'rank_by_agg_emb_sim' in selected_group.columns:
    rank_corr_distance_agg = selected_group[['rank_by_distance', 'rank_by_agg_emb_sim']].corr(method='spearman').iloc[0, 1]
    print(f"Spearman rank correlation between distance and aggregated embedding similarity ranks: {rank_corr_distance_agg:.4f}")
else:
    print("\nRank columns are not available for correlation computation between distance and aggregated embedding similarity.")

# Find similar dimensions (for demonstration)
dimension_df = selected_group[[
    'label',

    'sku_first', 'sku_second',

    # 'options_first', 'options_second',

    'width_first', 'height_first',
    'width_second', 'height_second',

    # 'balance_first',
    'balance_second',

    'sales_first',
    'sales_second',

    'rank_by_distance',
    'rank_by_similarity',
    # 'rank_by_wh_emb_sim',
    'rank_by_agg_emb_sim',

    # 'euclidean_distance',
    # 'options_similarity',
    # 'wh_emb_sim',
    # 'agg_emb_sim',
]].copy()

filtered_dimension_df = dimension_df

# Sort the filtered results by rank_by_similarity

# sort_rank_col = 'rank_by_similarity'
# sort_rank_col = 'rank_by_distance'
sort_rank_col = 'rank_by_agg_emb_sim'

filtered_dimension_df = filtered_dimension_df.sort_values(sort_rank_col, ascending=True)

pd.options.display.float_format = '{:.2f}'.format
filtered_dimension_df = filtered_dimension_df.reset_index(drop=True)

# Display the filtered and sorted results
pd.set_option('display.max_colwidth', None)
print("\nProducts with extracted dimensions, ranks, embedding similarity, and labels:")

display(filtered_dimension_df[filtered_dimension_df.label == 1])
display(filtered_dimension_df)



Reference product dimensions: Height 107 cm, Width 157 cm

Products sorted by Euclidean distance to reference dimensions:

Spearman rank correlation between distance and similarity ranks: 0.0640
Spearman rank correlation between distance and width/height embedding similarity ranks: 0.5135
Spearman rank correlation between distance and aggregated embedding similarity ranks: 0.9319

Products with extracted dimensions, ranks, embedding similarity, and labels:


,label,sku_first,sku_second,width_first,height_first,width_second,height_second,balance_second,sales_first,sales_second,rank_by_distance,rank_by_similarity,rank_by_agg_emb_sim
2,1,64354767,182755008,157,107,160,100,583,271,2117,2.00,28.00,3.00
3,1,64354767,182755012,157,107,160,100,203,271,899,3.00,29.00,4.00
4,1,64354767,182755006,157,107,160,100,0,271,320,4.00,30.00,5.00


,label,sku_first,sku_second,width_first,height_first,width_second,height_second,balance_second,sales_first,sales_second,rank_by_distance,rank_by_similarity,rank_by_agg_emb_sim
0,0,64354767,72587452,157,107,160,90,76,271,701,5.00,24.00,1.00
1,0,64354767,147848541,157,107,160,100,145,271,2354,1.00,27.00,2.00
2,1,64354767,182755008,157,107,160,100,583,271,2117,2.00,28.00,3.00
3,1,64354767,182755012,157,107,160,100,203,271,899,3.00,29.00,4.00
4,1,64354767,182755006,157,107,160,100,0,271,320,4.00,30.00,5.00
5,0,64354767,78583225,157,107,160,85,1,271,1069,10.00,5.00,6.00
6,0,64354767,78583224,157,107,160,85,22,271,614,11.00,4.00,7.00
7,0,64354767,158564491,157,107,160,85,46,271,429,12.00,2.00,8.00
8,0,64354767,157384922,157,107,160,85,30,271,320,13.00,3.00,9.00
9,0,64354767,16863976,157,107,150,90,47,271,1077,6.00,17.00,10.00
